# LabQueue API Scripts

In [ ]:
from wsse.client.requests.auth import WSSEAuth
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint

In [ ]:
username = "" # e.g. "cas-princeton-atli"
API_SECRET = ""
wsse_auth = WSSEAuth(username, API_SECRET)
base_url = "https://www.labqueue.io/api/v1/"

### Check a single Lab TA's requests

In [ ]:
# get the requests that were accepted by a particular Lab TA
netid = ""
# time must be like the following format "yyyy-mm-ddThh:mm"
time = "2022-09-01T00:00"

payload = {"accepted_by": netid, "accepted_after": time, }

result = requests.get(base_url + "requests/query/", auth=wsse_auth, params=payload)
pprint.pprint(result.json())

In [ ]:
# for specifying more specifics such as start and end or course
curr_day_start = "2022-03-20T20:00"
curr_day_end = "2022-03-20T22:00"

# payload = {"created_after": curr_day_start, "created_before": curr_day_end, "course": "217"}
payload = {"created_after": curr_day_start, "created_before": curr_day_end, "accepted_by": netid}
result = requests.get(base_url + "requests/query/", auth=wsse_auth, params=payload)
pprint.pprint(result.json())

### Look at the distribution of requests across all Lab TAs

In [ ]:
# look at the number of requests accepted across all Lab TAs

roster = requests.get(base_url + "queues/intro-cs-lab/roster/", auth=wsse_auth)
time = "2022-09-01T00:00"

num_requests = []

for lab_ta in roster.json()["results"]:
    # query for the accpeted requests
    payload = {"accepted_by": lab_ta["netid"], "accepted_after": time}
    
    result = requests.get(base_url + "requests/query/", auth=wsse_auth, params=payload)
    
    num_requests.append(result.json()["count"])

print(num_requests)
    
plt.hist(num_requests)
plt.show()

### Look at the number of requests across day of the week

In [ ]:
# look at the number of requests across the day of the week
import datetime
start_date = datetime.date(year=2022, month=9, day=1)
end_date = datetime.date(year=2022, month=10, day=16)
delta = datetime.timedelta(days=1)

# make sure to separate 126, 226/217
# index 0 is monday, 1 is tuesday, ..., 6 is sunday
day_of_week_count = {"126": [0]*7, "226": [0]*7, "217": [0]*7}

# loop from start_date to end_date
curr_day = start_date
while curr_day <= end_date:
    print(curr_day)
    
    curr_day_start = curr_day.strftime("%Y-%m-%dT00:00")
    curr_day_end = curr_day.strftime("%Y-%m-%dT23:59")

    # get the requests on that day
    # do each course separately
    for course_name in ["126", "226", "217"]:
        payload = {"created_after": curr_day_start, "created_before": curr_day_end, "course": course_name}
        result = requests.get(base_url + "requests/query/", auth=wsse_auth, params=payload)
        
        day_of_week_count[course_name][curr_day.weekday()] += result.json()["count"]

    
    curr_day += delta

print(day_of_week_count)

In [ ]:
labels = ("Mon", "Tues", "Wed", "Thurs", "Fri", "Sat", "Sun")
plt.bar(np.arange(len(labels)), day_of_week_count["126"], align="center")
plt.xticks(np.arange(len(labels)), labels)
plt.ylabel("Count")
plt.title("COS126 Lab Requests by Day of Week")
plt.show()

In [ ]:
day_of_week_count_2xx = np.array(day_of_week_count["226"]) + np.array(day_of_week_count["217"])

plt.bar(np.arange(len(labels)), day_of_week_count_2xx, align="center")
plt.xticks(np.arange(len(labels)), labels)
plt.ylabel("Count")
plt.title("COS2xx Lab Requests by Day of Week")
plt.show()